# F-тест: совместная значимость

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования

from scipy.stats import f # f-распределение и критические значения

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Совместная значимость</span> </b><br>

Для линейной регрессии

$$
	(\log)y=\beta_0+\beta_1(\log)x_1+\cdots+\beta_k(\log)x_k+u=x^\top \beta+u
$$

Тестируется гипотеза, что несколько коэффициентов совместно равны нулю в генеральной совокупности. Например, последние $J$ коэффициентов

$$
H_0:\beta_{k-J+1}=\cdots=\beta_k=0
$$

__Интерпретация__: включённые в модель регрессоры совместно не влияют на зависимую переменную (в генеральной совокупности)

__Тестовая статистика__ для выбранной оценки ковариационной матрицы $\hat{V}$ формула для F-статистики (всё вычисляется автоматически!)

$$
\begin{aligned}
	F&=\frac{1}{J}(R\widehat{\beta})^\top
	\Bigl(R\cdot\hat{V}\cdot R^\top\Bigr)^{-1}
	(R\widehat{\beta}) &
	\underset{J \times (k+1)}{R}&=
	\begin{pmatrix} 
	0 & \cdots & 0 & 1 & 0 & \cdots & 0 \\ 
	0 & \cdots & 0 & 0 & 1 & \cdots & 0 \\
	\vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots \\
	0 & \cdots & 0 & 0 & 0 & \cdots & 1
	\end{pmatrix}=\begin{pmatrix} 0_{J\times (k+1-J)} & I_{J\times J}\end{pmatrix}
\end{aligned}
$$

__Критическое значение__: $F$-распределения со степенями свободы ($k$ - число объясняющих переменных)

$$ 
\begin{aligned} 
	df_1&=dfn=J & df_2&=dfd=n-k-1
\end{aligned}
$$

__Замечание__: 
* для F-тест есть неробастный (по умолчанию) и робастный вариант
* для подогнанной регрессии используем метод `.f_test(r_matrix)` или `.wald_test(r_matrix, use_f=True)`
* тестируемую гипотезу задаём в виде `'x1=x2=0'` или `'x1=0,x2=0'` (параметр `r_matrix`) 

__Вывод__

**<span style="color:purple">Способ 1:</span>** используем $F$-статистику

* Отвергаем гипотезу $H_0$ при $F>F_{cr}$, <i><b><span style="color: purple">совместное влияния значимо</span></b><br>
* Не отвергаем гипотезу $H_0$ при $F<F_{cr}$, <i><b><span style="color: purple">совместное влияния незначимо</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем $P-value$

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">совместное влияния значимо</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">совместное влияния незначимо</span> </b><br>

</div>

Для набора данных `sleep75` рассмотрим линейную регрессию __sleep~totwrk+age+male+south+smsa+yngkid+marr+union__

Тестируем совместную значимость __smsa, yngkid, marr, union__

$$
	H_0:\beta_{smsa}=\beta_{yngkid}=\beta_{marr}=\beta_{union}=0
$$

Для этой гипотезы $J=4$

In [ ]:
# импорт данных
df = pd.read_csv('sleep75.csv')

## F-тест (неробастный)

Используем OLS-оценку ковариационной матрицы $\hat{V}=s^2(X^\top X)^{-1}$

In [ ]:
# спецификация модели через формулу
mod = ols(formula='sleep~1+totwrk+age+male+south+smsa+yngkid+marr+union', data=df)
# подгонка модели с неробастной оценкой ковариационной матрицы
res_ols = mod.fit()

In [ ]:
F_test = res_ols.f_test('smsa=yngkid=marr=union=0')
print(F_test)

In [ ]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

In [ ]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.ppf(q=1-sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

In [ ]:
# степени свободы 
F_test.df_num, F_test.df_denom

In [ ]:
# альтернативно
W_test = res_ols.wald_test('smsa=0, yngkid=0, marr=0, union=0', use_f=True)
print(W_test)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">совместное влияния незначимо</span></b><br>

</div>

## F-тест (робастный)

Используем HC3-оценку ковариационной матрицы

In [ ]:
# спецификация модели через формулу
mod = ols(formula='sleep~1+totwrk+age+male+south+smsa+yngkid+marr+union', data=df)
# подгонка модели с робастной HC3-оценкой ковариационной матрицы
res_hc = mod.fit(cov_type='HC3')

In [ ]:
# альтернативная спецификация тестируемой гипотезы
F_test = res_hc.f_test('smsa=0, yngkid=0, marr=0, union=0')
print(F_test)

In [ ]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

In [ ]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.isf(q=sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

In [ ]:
# альтернативно
W_test = res_hc.wald_test('smsa=0, yngkid=0, marr=0, union=0', use_f=True)
print(W_test)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">совместное влияния незначимо</span></b><br>

</div>